1. Utilizando Vistas

1.1. Accediendo a la base de datos TSQL

USE TSQL;
GO

1.2 Seleccionar y ejecutar el siguiente código para crear una vista

In [2]:
CREATE VIEW HR.EmpPhoneList
AS
SELECT empid, lastname, firstname, phone
FROM HR.Employees;
GO

Commands completed successfully.

Total execution time: 00:00:00.186

1.3 Consultar la nueva vista

In [3]:
SELECT empid, lastname, firstname, phone
FROM HR.EmpPhoneList;
GO

(9 rows affected)

Total execution time: 00:00:00.220

empid,lastname,firstname,phone
1,Davis,Sara,(206) 555-0101
2,Funk,Don,(206) 555-0100
3,Lew,Judy,(206) 555-0103
4,Peled,Yael,(206) 555-0104
5,Buck,Sven,(71) 234-5678
6,Suurs,Paul,(71) 345-6789
7,King,Russell,(71) 123-4567
8,Cameron,Maria,(206) 555-0102
9,Dolgopyatova,Zoya,(71) 456-7890


1.4 Crear una vista ustilizando un JOIN multi-tabla

In [4]:
CREATE VIEW Sales.OrdersByEmployeeYear
AS
    SELECT  emp.empid AS employee ,
            YEAR(ord.orderdate) AS orderyear ,
            SUM(od.qty * od.unitprice) AS totalsales
    FROM    HR.Employees AS emp
            JOIN Sales.Orders AS ord ON emp.empid = ord.empid
            JOIN Sales.OrderDetails AS od ON ord.orderid = od.orderid
    GROUP BY emp.empid ,
            YEAR(ord.orderdate)
GO

Commands completed successfully.

Total execution time: 00:00:00.187

1.5 Seleccionar la vista

In [5]:
SELECT employee, orderyear, totalsales
FROM Sales.OrdersByEmployeeYear
ORDER BY employee, orderyear;

(27 rows affected)

Total execution time: 00:00:00.265

employee,orderyear,totalsales
1,2006,38789.0000
1,2007,97533.5800
1,2008,65821.1300
2,2006,22834.7000
2,2007,74958.6000
2,2008,79955.9600
3,2006,19231.8000
3,2007,111788.6100
3,2008,82030.8900
4,2006,53114.8000


1.6 Limpiar los cambios

In [6]:
DROP VIEW Sales.OrdersByEmployeeYear;
DROP VIEW HR.EmpPhoneList;

Commands completed successfully.

Total execution time: 00:00:00.256

2. Utilizando Funciones de Tabla en Linea

2.1. Ejecutar y consultar la siguiente función de ejemplo dbo.GetNums() toma como parámetros: @low (bigint) y @high (bigint)

In [7]:
SELECT * FROM dbo.GetNums(10,20);
GO

(11 rows affected)

Total execution time: 00:00:00.331

n
10
11
12
13
14
15
16
17
18
19


2.2. Crear una función para calcular para extensión de líneas para ordenes de compra

In [8]:
CREATE FUNCTION Sales.fn_LineTotal ( @orderid INT )
RETURNS TABLE
AS
RETURN
    SELECT  orderid, productid, unitprice, qty, discount,
            CAST(( qty * unitprice * ( 1 - discount ) ) AS DECIMAL(8, 2)) AS line_total
    FROM    Sales.OrderDetails
    WHERE   orderid = @orderid ;
GO

Commands completed successfully.

Total execution time: 00:00:00.263

2.3. Utilizando la función

In [9]:
SELECT orderid, productid, unitprice, qty, discount, line_total
FROM Sales.fn_LineTotal(10252) AS LT;
GO

(3 rows affected)

Total execution time: 00:00:00.300

orderid,productid,unitprice,qty,discount,line_total
10252,20,64.8000,40,0.050,2462.40
10252,33,2.0000,25,0.050,47.50
10252,60,27.2000,40,0.000,1088.00


2.4. Limpiar los cambios

In [10]:
DROP FUNCTION Sales.fn_LineTotal;
GO

Commands completed successfully.

Total execution time: 00:00:00.257

3. Utilizando Tablas Derivadas

3.1. Visualizando alias internos de columnas.

In [11]:
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM (SELECT YEAR(orderdate) AS orderyear, custid
FROM Sales.Orders) AS derived_year
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.315

orderyear,cust_count
2006,67
2007,86
2008,81


3.2. Utilizando una variable como parametro de una tabla derivada

In [12]:
DECLARE @emp_id INT = 9;
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM (	
	SELECT YEAR(orderdate) AS orderyear, custid
	FROM Sales.Orders
	WHERE empid=@emp_id
) AS derived_year
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.288

orderyear,cust_count
2006,5
2007,16
2008,16


3.3. Enlazando tablas derivadas

In [13]:
SELECT orderyear, cust_count
FROM  (
	SELECT  orderyear, COUNT(DISTINCT custid) AS cust_count
	FROM (
		SELECT YEAR(orderdate) AS orderyear ,custid
        FROM Sales.Orders) AS derived_table_1
	GROUP BY orderyear) AS derived_table_2
WHERE cust_count > 80;

(2 rows affected)

Total execution time: 00:00:00.303

orderyear,cust_count
2007,86
2008,81


3.4. Una alternativa al ejemplo anterior sería.

In [14]:
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
	FROM (
		SELECT YEAR(orderdate) AS orderyear ,custid
        FROM Sales.Orders) AS derived_table_1
	GROUP BY orderyear
HAVING COUNT(DISTINCT custid) > 80;

(2 rows affected)

Total execution time: 00:00:00.245

orderyear,cust_count
2007,86
2008,81


4. Utilizando Expresions Comunes de Tabla

4.1. Ejecutar la siguiente expresión.

In [15]:
WITH CTE_year AS
	(
	SELECT YEAR(orderdate) AS orderyear, custid
	FROM Sales.Orders
	)
SELECT orderyear, COUNT(DISTINCT custid) AS cust_count
FROM CTE_year
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.218

orderyear,cust_count
2006,67
2007,86
2008,81


4.2. También se puede utilizar de modo recursico

In [16]:
WITH EmpOrg_CTE AS
(SELECT empid, mgrid, lastname, firstname --anchor query
	FROM HR.Employees
WHERE empid = 5 -- starting "top" of tree. Change this to show other root employees

UNION ALL
SELECT child.empid, child.mgrid, child.lastname, child.firstname -- recursive member which refers back to CTE
	FROM EmpOrg_CTE AS parent
	JOIN HR.Employees AS child
	ON child.mgrid=parent.empid
)
SELECT empid, mgrid, lastname, firstname
FROM EmpOrg_CTE;

(4 rows affected)

Total execution time: 00:00:00.243

empid,mgrid,lastname,firstname
5,2,Buck,Sven
6,5,Suurs,Paul
7,5,King,Russell
9,5,Dolgopyatova,Zoya
